In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install tensorflow keras pandas opencv-python numpy matplotlib seaborn

In [ ]:
import keras
from keras.models import *
from keras.layers import *
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from keras import optimizers
from sklearn.utils import shuffle
import seaborn as sns
import cv2, os
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dropout

In [ ]:
sns.set_style("whitegrid", {'axes.grid' : False})

In [ ]:
# Define the path to the dataset folders

dir_data="/content/drive/MyDrive/FYP/Datasets/Plantsdatasets"
dir_seg = dir_data + "/images"
dir_img = dir_data + "/masks/"

In [ ]:
def load_images(data_path, label_path):
    data_images = []
    label_images = []
    for filename in sorted(os.listdir(data_path)):
        if filename.endswith('.png'):
            # Load data image
            data_image = Image.open(os.path.join(data_path, filename)).convert('L')
            data_images.append(np.array(data_image))

            # Load label image
            label_image = Image.open(os.path.join(label_path, filename)).convert('L')
            label_images.append(np.array(label_image))
    return np.array(data_images), np.array(label_images)


In [ ]:
dir_seg = "/content/drive/MyDrive/FYP/Datasets/Plantsdatasets/masks"
dir_img = "/content/drive/MyDrive/FYP/Datasets/Plantsdatasets/images"

# Get the list of files in each directory
ldseg = np.array(os.listdir(dir_seg))
ldimg = np.array(os.listdir(dir_img))

# Pick the first image file
fnm = ldseg[0]
fnmg = ldimg[0]
print(fnm)

# Read in the original image and segmentation labels
seg = cv2.imread(os.path.join(dir_seg, fnm))
img_is = cv2.imread(os.path.join(dir_img, fnmg))

if seg is None or img_is is None:
    raise FileNotFoundError("One or more image files not found.")

print("Segmented Image shape:", seg.shape)
print("Original Image Shape:", img_is.shape)

# Resize the images to 256x256
seg = cv2.resize(seg, (256, 256))
img_is = cv2.resize(img_is, (256, 256))

print("After:\n")
print("Segmented Image shape:", seg.shape)
print("Original Image Shape:", img_is.shape)

mi, ma = np.min(seg), np.max(seg)
n_classes = 1  # Set to 1 for binary segmentation (one class)
print("Minimum seg = {}, Maximum seg = {}, Total number of segmentation classes = {}".format(mi, ma, n_classes))

print("*****************************")
# Resize the images to 224x224
seg = cv2.resize(seg, (224, 224))
img_is = cv2.resize(img_is, (224, 224))
# Convert to binary format
seg = (seg > 0).astype(np.uint8)
print("Resized seg.shape={},\nResized img_is.shape={}".format(seg.shape, img_is.shape))


In [ ]:
fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(1, 1, 1)
# Assuming img_is is defined somewhere in your code before the plotting

ax.imshow(img_is)
ax.set_title("original image")
plt.show()

fig = plt.figure(figsize=(15, 10))
for k in range(2):
    ax = fig.add_subplot(3, 4, k + 1)
    ax.imshow((seg == k) * 1.0)
    ax.set_title("Segmented Image\nlabel = {}".format(k))

plt.show()

In [ ]:
fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(1, 1, 1)
ax.imshow((seg == 1) * 1.0)
ax.set_title("Segmented Image\nlabel = 1")
plt.show()


In [ ]:
#this is for coloring the segments
def give_color_to_seg_img(seg, n_classes):
    if len(seg.shape) == 3:
        seg = seg[:, :, 0]
    seg_img = np.zeros((seg.shape[0], seg.shape[1], 3)).astype('float')
    colors = sns.color_palette("hls", n_classes)

    for c in range(n_classes):
        segc = (seg == c)
        seg_img[:, :, 0] += (segc * (colors[c][0]))
        seg_img[:, :, 1] += (segc * (colors[c][1]))
        seg_img[:, :, 2] += (segc * (colors[c][2]))

    return seg_img


In [ ]:
input_height, input_width = 224, 224
output_height, output_width = 224, 224

ldseg = np.array(os.listdir(dir_seg))
for fnm in ldseg[np.random.choice(len(ldseg), 3, replace=False)]:
    fnm = fnm.strip()  # Remove any leading/trailing whitespace
    seg_path = os.path.join(dir_seg, fnm)
    img_path = os.path.join(dir_img, fnm)

    # Check if file paths are valid
    if not os.path.exists(seg_path) or not os.path.exists(img_path):
        print(f"File not found: {seg_path} or {img_path}")
        continue

    seg = cv2.imread(seg_path)
    img_is = cv2.imread(img_path)

    # Check if images are read correctly
    if seg is None or img_is is None:
        print(f"Failed to read images for {fnm}")
        continue

    seg_img = give_color_to_seg_img(seg, n_classes)
    #seg = seg.astype(np.uint8)  # Convert seg_img to uint8 dtype
    #seg = seg / 255.0  # Normalize seg_img to [0, 1]

    fig = plt.figure(figsize=(20, 40))
    ax = fig.add_subplot(1, 4, 1)
    ax.imshow(seg)

    ax = fig.add_subplot(1, 4, 2)
    ax.imshow(img_is)
    ax.set_title("original image {}".format(img_is.shape[:2]))

    ax = fig.add_subplot(1, 4, 3)
    ax.imshow(cv2.resize(seg, (output_height, output_width)))

    ax = fig.add_subplot(1, 4, 4)
    ax.imshow(cv2.resize(img_is, (output_height, output_width)) )
    ax.set_title("resized to {}".format((output_height, output_width)))
    plt.show()


In [ ]:
seg_img.shape

In [ ]:
import cv2
import numpy as np

def getImageArr(path, width, height):
    img = cv2.imread(path)
    img = cv2.resize(img, (width, height))
    # Perform any additional preprocessing if needed
    return img

def getSegmentationArr(path, num_classes, width, height):
    seg = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    seg = cv2.resize(seg, (width, height))
    seg = (seg > 0).astype(np.uint8)  # Convert to binary format
    # Perform any additional preprocessing if needed
    return seg




In [ ]:
def FCN8(nClasses, input_height=224, input_width=224):
    assert input_height % 32 == 0
    assert input_width % 32 == 0
    IMAGE_ORDERING = "channels_last"

    img_input = Input(shape=(input_height, input_width, 3))

    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1', data_format=IMAGE_ORDERING)(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2', data_format=IMAGE_ORDERING)(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool', data_format=IMAGE_ORDERING)(x)
    f1 = x

    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1', data_format=IMAGE_ORDERING)(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2', data_format=IMAGE_ORDERING)(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool', data_format=IMAGE_ORDERING)(x)
    f2 = x

    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1', data_format=IMAGE_ORDERING)(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2', data_format=IMAGE_ORDERING)(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3', data_format=IMAGE_ORDERING)(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool', data_format=IMAGE_ORDERING)(x)
    pool3 = x

    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1', data_format=IMAGE_ORDERING)(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2', data_format=IMAGE_ORDERING)(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3', data_format=IMAGE_ORDERING)(x)
    pool4 = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool', data_format=IMAGE_ORDERING)(x)

    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1', data_format=IMAGE_ORDERING)(pool4)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2', data_format=IMAGE_ORDERING)(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3', data_format=IMAGE_ORDERING)(x)
    pool5 = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool', data_format=IMAGE_ORDERING)(x)

    n = 4096
    o = Conv2D(n, (7, 7), activation='relu', padding='same', name="conv6", data_format=IMAGE_ORDERING)(pool5)
    conv7 = Conv2D(n, (1, 1), activation='relu', padding='same', name="conv7", data_format=IMAGE_ORDERING)(o)

    conv7_4 = Conv2DTranspose(nClasses, kernel_size=(4, 4), strides=(4, 4), use_bias=False, data_format=IMAGE_ORDERING)(conv7)
    pool411 = Conv2D(nClasses, (1, 1), activation='relu', padding='same', name="pool4_11", data_format=IMAGE_ORDERING)(pool4)
    pool411_2 = Conv2DTranspose(nClasses, kernel_size=(2, 2), strides=(2, 2), use_bias=False, data_format=IMAGE_ORDERING)(pool411)
    pool311 = Conv2D(nClasses, (1, 1), activation='relu', padding='same', name="pool3_11", data_format=IMAGE_ORDERING)(pool3)
    o = Add(name="add")([pool411_2, pool311, conv7_4])
    o = Conv2DTranspose(nClasses, kernel_size=(8, 8), strides=(8, 8), use_bias=False, data_format=IMAGE_ORDERING)(o)
    o = Activation('sigmoid')(o)  # Use sigmoid activation for binary segmentation

    model = Model(img_input, o)

    return model

In [ ]:
model = FCN8(nClasses=n_classes, input_height=224, input_width=224)

In [ ]:
model.summary()


In [ ]:
images = os.listdir(dir_img)
images.sort()
segmentations = os.listdir(dir_seg)
segmentations.sort()

# Initialize empty lists
X = []
Y = []

# Ensure every image has a corresponding segmentation mask
for im in images:
    corresponding_seg = im  # Assuming filenames match between images and masks
    if corresponding_seg in segmentations:
        X.append(getImageArr(os.path.join(dir_img, im), input_width, input_height))
        Y.append(getSegmentationArr(os.path.join(dir_seg, corresponding_seg), n_classes, output_width, output_height))
    else:
        print(f"No corresponding segmentation found for image: {im}")

# Convert to arrays and check lengths
X = np.array(X)
Y = np.array(Y)

if X.shape[0] != Y.shape[0]:
    raise ValueError(f"The number of samples in X and Y do not match: {X.shape[0]} != {Y.shape[0]}")

train_rate = 0.70
index_train = np.random.choice(X.shape[0], int(X.shape[0] * train_rate), replace=False)
index_test = list(set(range(X.shape[0])) - set(index_train))

# Shuffle the data
X, Y = shuffle(X, Y)

# Split the data
X_train, y_train = X[index_train], Y[index_train]
X_test, y_test = X[index_test], Y[index_test]

print("Input Training Shape:", X_train.shape, "\nOutput Training Shape:", y_train.shape)
print("Input Testing Shape:", X_test.shape, "\nOutput Testing Shape: ", y_test.shape)

In [ ]:
# Define the callbacks
checkpoint = ModelCheckpoint("/content/drive/MyDrive/FYP/Trained Models/best_model.h5",
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             save_weights_only=False,
                             mode='min',
                             save_freq='epoch')
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min', restore_best_weights=True)


In [ ]:
# Compile the model
sgd = keras.optimizers.Adam(learning_rate=1e-4)
model.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [ ]:
# Train the model with callbacks
hist1 = model.fit(X_train, y_train,
                  validation_data=(X_test, y_test),
                  batch_size=32,
                  epochs=100,
                  verbose=1,
                  callbacks=[checkpoint, early_stopping])

In [ ]:
# Save the entire model including architecture and weights
model.save("/content/drive/MyDrive/FYP/Trained Models/best_model.h5")


In [ ]:
# Load the best weights
model.load_weights("/content/drive/MyDrive/FYP/Trained Models/best_model.h5")

In [ ]:
# Evaluate the model on the test set
evaluation = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Loss: {evaluation[0]}, Test Accuracy: {evaluation[1]}")

In [ ]:
# Plot Training and Validation Accuracy
plt.plot(hist1.history['accuracy'], label='Training Accuracy')
plt.plot(hist1.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Plot Loss
plt.subplot(2, 1, 1)
plt.plot(hist1.history['loss'], label='Loss')
plt.legend()

# Plot Accuracy
plt.subplot(2, 1, 2)
plt.plot(hist1.history['accuracy'], label='Accuracy')
plt.legend()

plt.show()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
for key in ['loss', 'accuracy']:
    plt.plot(hist1.history[key],label=key)
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Retrieve the training history
training_loss = hist1.history['loss']
training_accuracy = hist1.history['accuracy']
validation_loss = hist1.history['val_loss']
validation_accuracy = hist1.history['val_accuracy']

# Plot the training and validation loss
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(training_loss, label='Training Loss')
plt.plot(validation_loss, label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot the training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(training_accuracy, label='Training Accuracy')
plt.plot(validation_accuracy, label='Validation Accuracy')
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
y_pred = model.predict(X_test)
y_predi = (y_pred > 0.5).astype(np.uint8)  # Apply threshold for binary classification
y_testi = (y_test > 0.5).astype(np.uint8)

print(y_testi.shape, y_predi.shape)


In [ ]:
import numpy as np

def IoU(Yi, y_predi):
    # Mean Intersection over Union
    # Mean IoU = TP / (FN + TP + FP)

    IoUs = []
    Nclass = int(np.max(Yi)) + 1
    for c in range(Nclass):
        TP = np.sum((Yi == c) & (np.squeeze(y_predi) == c))
        FP = np.sum((Yi != c) & (np.squeeze(y_predi) == c))
        FN = np.sum((Yi == c) & (np.squeeze(y_predi) != c))
        IoU = TP / float(TP + FP + FN)
        print("class {:02.0f}: #TP={:6.0f}, #FP={:6.0f}, #FN={:5.0f}, IoU={:4.3f}".format(c, TP, FP, FN, IoU))
        IoUs.append(IoU)
    mIoU = np.mean(IoUs)
    print("_________________")
    print("Mean IoU: {:4.3f}".format(mIoU))

# Assuming y_testi and y_predi are the true labels and predicted labels, respectively
IoU(y_testi == 1, y_predi == 1)


In [ ]:
import numpy as np

def IoU(Yi, y_predi):
    # Mean Intersection over Union
    # Mean IoU = TP / (FN + TP + FP)

    IoUs = []
    Nclass = int(np.max(Yi)) + 1
    for c in range(Nclass):
        TP = np.sum((Yi == c) & (np.squeeze(y_predi) == c))
        FP = np.sum((Yi != c) & (np.squeeze(y_predi) == c))
        FN = np.sum((Yi == c) & (np.squeeze(y_predi) != c))
        IoU = TP / float(TP + FP + FN)
        print("class {:02.0f}: #TP={:6.0f}, #FP={:6.0f}, #FN={:5.0f}, IoU={:4.3f}".format(c, TP, FP, FN, IoU))
        IoUs.append(IoU)
    mIoU = np.mean(IoUs)
    print("_________________")
    print("Mean IoU: {:4.3f}".format(mIoU))

# Assuming y_testi and y_predi are the true labels and predicted labels, respectively
IoU(y_testi, y_predi)



In [ ]:
import matplotlib.pyplot as plt

shape = (224, 224)
n_classes = 1  # Set to 1 for binary segmentation (one class)

for i in range(10):
    img_is = (X_test[i]) * (255)
    seg = y_predi[i]
    segtest = y_testi[i]

    # Do not normalize the original image
    img_is_normalized = img_is

    # Normalize the predicted and true segmentation masks
    seg_normalized = (seg == 0).astype(float)
    segtest_normalized = (segtest == 0).astype(float)

    fig = plt.figure(figsize=(10, 30))
    ax = fig.add_subplot(1, 3, 1)
    ax.imshow(img_is_normalized)  # Ensure that the image is of 'uint8' type
    ax.set_title("Original")

    ax = fig.add_subplot(1, 3, 2)
    ax.imshow(seg_normalized)
    ax.set_title("Predicted Class")

    ax = fig.add_subplot(1, 3, 3)
    ax.imshow(segtest_normalized)
    ax.set_title("True Class")
    plt.show()


In [ ]:
shape = (224, 224)
n_classes = 1

for i in range(1):
    img_is = (X_test[i] ) * (255)
    seg = y_predi[i]
    segtest = y_testi[i]

    # Do not normalize the original image
    img_is_normalized = img_is

    fig = plt.figure(figsize=(10, 30))
    ax = fig.add_subplot(1, 3, 1)
    ax.imshow(img_is_normalized)  # Ensure that the image is of 'uint8' type
    ax.set_title("Original")

    ax = fig.add_subplot(1, 3, 2)
    ax.imshow(give_color_to_seg_img(seg, n_classes))
    ax.set_title("Predicted Class")

    ax = fig.add_subplot(1, 3, 3)
    ax.imshow(give_color_to_seg_img(segtest, n_classes))
    ax.set_title("True Class")
    plt.show()


In [ ]:
import cv2
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt

# Load the pre-trained model from the H5 file
model = keras.models.load_model("/content/drive/MyDrive/FYP/Trained Models/DiseasedPlantFCN.h5")

# Load and preprocess a new image for prediction
new_image_path = "/content/drive/MyDrive/FYP/Datasets/images/00010.png"
new_image = cv2.imread(new_image_path)
new_image = cv2.resize(new_image, (224, 224))  # Resize the image to match the model's input size
new_image = new_image / 255  # Normalize the image to [0, 1]

# Convert the image data type to uint8
new_image = (new_image * 255).astype(np.uint8)

# Expand dimensions to match the model's input shape (add batch dimension)
new_image = np.expand_dims(new_image, axis=0)

# Make predictions
predictions = model.predict(new_image)

# Assuming your model outputs binary segmentation
binary_predictions = (predictions < 0.5).astype(np.uint8)

# Visualize the results
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Display the original image
axes[0].imshow(new_image[0])
axes[0].set_title("Original Image")

# Display the predicted segmentation
axes[1].imshow(binary_predictions[0, :, :, 0], cmap="gray")
axes[1].set_title("Predicted Segmentation")

plt.show()


In [ ]:
import numpy as np


# Extract the specific channel (assuming it's the first channel)
segmentation_channel = binary_predictions[0, :, :, 0]

# Flatten the segmentation channel
flat_predictions = segmentation_channel.flatten()

# Count the number of pixels with value 1 and 0
count_ones = np.sum(flat_predictions == 1)
count_zeros = np.sum(flat_predictions == 0)

# Calculate the percentage of pixels with value 1 and 0
percentage_ones = (count_ones / flat_predictions.size) * 100
percentage_zeros = (count_zeros / flat_predictions.size) * 100

# Print the results
print(f"Number of pixels with value 1: {count_ones}")
print(f"Number of pixels with value 0: {count_zeros}")
print(f"Percentage of pixels with value 1: {percentage_ones:.2f}%")
print(f"Percentage of pixels with value 0: {percentage_zeros:.2f}%")

print(f"\n\nPlant is infected: {percentage_ones:.2f}%")

In [ ]:
from tensorflow.keras.models import load_model

# Load the trained model from the .h5 file
model = load_model("/content/drive/MyDrive/FYP/Trained Models/DiseasedPlantFCN.h5")

# Evaluate the model on the testing dataset
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"Testing Loss: {test_loss}, Testing Accuracy: {test_accuracy}")

# Evaluate the model on the training dataset
train_loss, train_accuracy = model.evaluate(X_train, y_train, verbose=1)
print(f"Training Loss: {train_loss}, Training Accuracy: {train_accuracy}")



In [ ]:
# Load the trained model from the .h5 file
model = load_model("/content/drive/MyDrive/FYP/Trained Models/best_model.h5")

# Evaluate the model on the testing dataset
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"Testing Loss: {test_loss}, Testing Accuracy: {test_accuracy}")

# Evaluate the model on the training dataset
train_loss, train_accuracy = model.evaluate(X_train, y_train, verbose=1)
print(f"Training Loss: {train_loss}, Training Accuracy: {train_accuracy}")
